In [ ]:
# we either import locally (after the repo was cloned)
# or we first clone it and then use it (e.g., when run in Google Colab)

shell = get_ipython().__class__.__name__ 

if shell == 'Shell':
    # imports when launched in e.g., Google Colab
    !git clone https://github.com/filipzz/athena.git r2b2
    from r2b2.code.athena.athena import AthenaAudit
    from r2b2.code.athena.election import Election
    from r2b2.code.athena.audit import Audit
else: # shell ==  'ZMQInteractiveShell' or shell == 'TerminalInteractiveShell'
    # local imports if you run it with e.g., Jupyter
    from athena.athena import AthenaAudit
    from athena.election import Election
    from athena.audit import Audit


In [ ]:
# risk limit of the audit
risk_limit = 0.1 

In [ ]:
# address of the file with election results
election_results = "https://raw.githubusercontent.com/gwexploratoryaudits/brla_explore/poorvi/R2Audits/Athena/Scripts/2020MontgomeryPrimary/2020_montgomery_formatted.json"


w = Audit("athena", risk_limit)

w.read_election_results(election_results)

w.get_contests()

In [ ]:
contest = "d_senator"

w.load_contest(contest)

w.election.print_election()

In [ ]:
w.election.print_election()

In [ ]:
w.run_interactive()

# This part is only needed if you are not planning to enter election details "by hand"


In [ ]:
audit_type = "athena" 

In [ ]:
logging.getLogger(__name__).setLevel("INFO")

In [ ]:
alpha = 0.1 # risk limit
name = "Election name"
winners = 1 # number of winners

In [ ]:
results = [17800, 9700] # ballots for candidates
total = 67000 # total number of ballots cast (can be set to 0)

In [ ]:
delta = 1
candidates = [string.ascii_uppercase[i] for i in range(len(results))]
ballots_cast = total
if ballots_cast < sum(results):
    ballots_cast = sum(results)
model = "bin"
round_schedule = []
election = {}
election["ballots_cast"] = ballots_cast
election["alpha"] = alpha
election["delta"] = delta
election["candidates"] = candidates
election["results"] = results
election["winners"] = winners
election["name"] = name
election["model"] = model
election_object = Election(election)
election_object.print_election()

In [ ]:
w = Audit(audit_type, alpha, delta)
w.add_election(election)
w.add_round_schedule(round_schedule)

w.run_interactive()
